In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('training_data_en.csv',sep=';',on_bad_lines='skip')

In [3]:
df_train

,ID,Text,Question,Answer
0,3335,"Nationwide is in robust financial health, havi...",What is the effect of achieving profits of ove...,Nationwide is in robust financial health
1,3336,All the Directors are resident in the UK and t...,What is the impact of the Company's small size...,the Board considers it sensible for all the Di...
2,3337,"Overall, Group trading continues to be subdued...",What is the main reason why the Group trading ...,legacy issues
3,3341,Included within the Group's administrative exp...,What is the effect of the Group taking a contr...,consolidate the results of a portfolio company...
4,3343,The following points were noted: • A review of...,What happened because of management performing...,The outstanding receivables are fully impaired
...,...,...,...,...
1991,5094,"Because the ""curing"" of concrete takes place o...","What are the consequences of the ""curing"" of c...",some of these tests span periods up to 12 mont...
1992,3568.a,Finally it is worth noting that in addition to...,What was the impact of the continuing enhanced...,has made us the biggest in our industry
1993,5044.1,The Group will operate in highly competitive m...,What explains the rapid evolution and adaptati...,"factors such as consumer tastes, trends and te..."
1994,1463,Any increases to Non-Executive Director fees w...,What factors will influence the decision to in...,the outcome of a review process and taking int...


In [4]:
df_train.loc[2]['Text']

'Overall, Group trading continues to be subdued in large part due to legacy issues'

In [5]:
# Same Text multiple cause and Effects

In [6]:
df_grouped = df_train.groupby('Text').count()  # Group by 'Text' and get counts
filtered_df = df_grouped[df_grouped['ID'] > 1]  # Replace 'some_column' with the column you're counting

In [7]:
import os
import openai
from openai import OpenAI
# get client

# choice = input('Willing to use paid or free?')

# if choice=='paid':
#     # paid, get your key and add in key.env file
#     client = OpenAI(api_key=OAI_KEY)
# else:
#     # free, get your free key and add in key.env file
#     client = openai.OpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     # create your api key https://console.groq.com/keys
#     api_key="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"
#     )

In [8]:
client = openai.OpenAI(
    base_url="http://127.0.0.1:1337/v1",
    # base_url="http://localhost:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

In [9]:
# List available models
models = client.models.list()

In [10]:
[i.id for i in models.data]

['Llama-3.2-3B-Instruct-Q3_K_L',
 'Meta-Llama-3.1-70B-Instruct-IQ1_M',
 'Meta-Llama-3.1-70B-Instruct-IQ2_XXS',
 'Meta-Llama-3.1-70B-Instruct-Q4_K_M',
 'Meta-Llama-3.1-8B-Instruct-Q8_0',
 'aya-23-35b',
 'aya-23-8b',
 'bakllava-1',
 'claude-3-5-sonnet-20240620',
 'claude-3-haiku-20240307',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'codeninja-1.0-7b',
 'codestral-22b',
 'command-r',
 'command-r-34b',
 'command-r-plus',
 'deepseek-coder-1.3b',
 'deepseek-coder-34b',
 'deepseek-coder-7b-instruct-v1.5-Q4_K_M',
 'gemma-1.1-2b-it',
 'gemma-1.1-7b-it',
 'gemma-2-27b-it',
 'gemma-2-2b-it',
 'gemma-2-9b-it',
 'gemma-7b-it',
 'gpt-3.5-turbo',
 'gpt-4-turbo',
 'gpt-4-vision-preview',
 'gpt-4o',
 'llama2-chat-70b',
 'llama2-chat-7b',
 'llama3-70b-8192',
 'llama3-8b-8192',
 'llama3-8b-instruct',
 'llama3-hermes-8b',
 'llama3.1-70b-instruct',
 'llama3.1-8b-instruct',
 'llama3.2-1b-instruct',
 'llama3.2-3b-instruct',
 'llamacorn-1.1b',
 'llava-13b',
 'llava-7b',
 'mistral-ins-7b-q4',
 'm

In [11]:
# List of models
# https://community.make.com/t/what-is-the-difference-between-system-user-and-assistant-roles-in-chatgpt/36160
# Free:- https://console.groq.com/docs/models
# Paid:- https://openai.com/api/pricing/


def get_predicted_linker(cntxt,qstn):
    completion = client.chat.completions.create(
        model= 'gemma-2-9b-it', #'llama-3.1-8b-instant',# chatgpt cheapest and efficient -> "gpt-4o-mini",
        stop= ["<start_of_turn>","<end_of_turn>","\n"], #["<|eot_id|>", "\n",'</s>'],
        messages=[
            {"role": "system", "content": """
            You are an AI assistant in Finance Causal generation. Your task is to The answer the cause or effect previously questioned, extracted verbatim from the text, making it extractive.
            Here are some examples:
            
            Example 1:
            CONTEXT: Nationwide is in robust financial health, having achieved profits of over £1 billion for the third consecutive year. 
            As a mutual, profits are not the only barometer of our success, but they are important because they allow us to maintain our financial strength, to invest with confidence, 
            and to return value to you, our members, through pricing and service.When given new sequences, respond with only the predicted Linker SMILES and nothing else.
            QUESTION: What is the effect of achieving profits of over £1 billion for the third consecutive year?
            ANSWER: Nationwide is in robust financial health

            Example 2:
            CONTEXT: All the Directors are resident in the UK and their biographical details, which are set out on page s 26 and 27, 
            demonstrate the wide range of skills and experience that they bring to the Board. In view of the Company's size and as the Board is comprised of only five Directors, 
            all of whom are independent, the Board considers it sensible for all the Directors to be members of the Audit Committee and of the Nomination and Remuneration Committee.
            QUESTION: What is the impact of the Company's small size and having a Board comprised of only five independent Directors?
            ANSWER: the Board considers it sensible for all the Directors to be members of the Audit Committee and of the Nomination and Remuneration Committee

            When given new Context and Question,just respond with only extracted verbatim from the ###Context as Answer and Do not add or modify anything.
            """},

            {"role": "user", "content": f"""
            ### Instruction
            You will be given financial text as context in ### Context.

            Read and understand definition of Cause and Effect:
            The cause and effect is defined as a relation established between two events, where the first
            event acts as the cause of the second event and the second event is the effect of the first event.
            One cause can have several effects. A cause is why an event happens. The effect is an event
            that happens because of cause. The cause and effect occurs based on the following criteria,
            where cause has to occur before effect, and whenever the cause occurs the effect has to occur.
 
            ### Context: {cntxt}
            ### Question: {qstn}
            ### Answer:
            """}
        ]
    )
    return completion.choices[0].message.content

In [12]:
t = df_train['Text'][5]
q = df_train['Question'][5]

In [13]:
df_train['Answer'][5]

'This year, we continued to implement the objectives set out in our Environmental Policy'

In [14]:
t

'This year, we continued to implement the objectives set out in our Environmental Policy and as a result, the Bank has introduced many initiatives to enhance our commitment to conducting our business'

In [15]:
q

'Why has the Bank introduced many initiatives to enhance their commitment to conducting their business?'

In [16]:
a = get_predicted_linker(t,q)

In [17]:
a

'This year, we continued to implement the objectives set out in our Environmental Policy  '

In [18]:
print(a)

This year, we continued to implement the objectives set out in our Environmental Policy  


In [19]:
df_train['Question'].apply(lambda x:x.split()[0]).unique()

array(['What', 'Why', 'In', 'Why,', 'According', 'Following', 'For',
       'Which', 'Given', 'Apart', 'To'], dtype=object)

In [20]:
from tqdm.notebook import tqdm

In [21]:
df = df_train

In [22]:
df['Predicted_Answer'] = None
for line,row in tqdm(df.iterrows()):
    context = row['Text']
    qstn = row['Question']
    a = get_predicted_linker(context,qstn)
    df.at[line, 'Predicted_Answer'] = a

0it [00:00, ?it/s]

In [23]:
# generate all causes and respective effect
# Roberta + decoder
# prompt tuning
# change topk top P , temp etc..

In [24]:
df

,ID,Text,Question,Answer,Predicted_Answer
0,3335,"Nationwide is in robust financial health, havi...",What is the effect of achieving profits of ove...,Nationwide is in robust financial health,Nationwide is in robust financial health
1,3336,All the Directors are resident in the UK and t...,What is the impact of the Company's small size...,the Board considers it sensible for all the Di...,the Board considers it sensible for all the Di...
2,3337,"Overall, Group trading continues to be subdued...",What is the main reason why the Group trading ...,legacy issues,legacy issues
3,3341,Included within the Group's administrative exp...,What is the effect of the Group taking a contr...,consolidate the results of a portfolio company...,and hence consolidate the results of a portfol...
4,3343,The following points were noted: • A review of...,What happened because of management performing...,The outstanding receivables are fully impaired,"As a result of the above actions, management c..."
...,...,...,...,...,...
1991,5094,"Because the ""curing"" of concrete takes place o...","What are the consequences of the ""curing"" of c...",some of these tests span periods up to 12 mont...,some of these tests span periods up to 12 mont...
1992,3568.a,Finally it is worth noting that in addition to...,What was the impact of the continuing enhanced...,has made us the biggest in our industry,the biggest in our industry and therefore inev...
1993,5044.1,The Group will operate in highly competitive m...,What explains the rapid evolution and adaptati...,"factors such as consumer tastes, trends and te...","factors such as consumer tastes, trends and te..."
1994,1463,Any increases to Non-Executive Director fees w...,What factors will influence the decision to in...,the outcome of a review process and taking int...,the outcome of a review process and taking int...


In [25]:
import pandas as pd
import numpy as np
df.to_csv('prediction_gemma-2-9b-it.csv',index=False)

In [26]:
from sentence_transformers import SentenceTransformer, util
from collections import namedtuple

model = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
st_model = SentenceTransformer(model)

Task2Data = namedtuple('Task2Data', ['index', 'context', 'question', 'answer'])

def SAS(predicted_answers, reference_answers):
    predictions_embeddings = st_model.encode(predicted_answers, convert_to_tensor=True)
    reference_embeddings = st_model.encode(reference_answers, convert_to_tensor=True)
    similarity_scores = [util.cos_sim(p, l).cpu().numpy() for p, l in zip(predictions_embeddings, reference_embeddings)]
    return np.mean(similarity_scores)

def ExactMatch(predicted_answers, reference_answers):
    return np.mean([1 if pred.lower().strip() == ref.lower().strip() else 0 for pred, ref in zip(predicted_answers, reference_answers)])


/home/sarang/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-23 07:28:13.185722: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [27]:
reference_answers = df['Answer'].values
predicted_answers = df['Predicted_Answer'].values

In [28]:
SAS(predicted_answers, reference_answers)

0.89748996

In [29]:
ExactMatch(predicted_answers, reference_answers)

0.28707414829659317